In [17]:
import pandas as pd
import requests
from datetime import datetime

In [18]:
def remove_multiindex(df:pd.DataFrame):
	return df.droplevel(level=0,axis=1)

In [19]:
# link for extract html data
def getdata(url):
    r = requests.get(url)
    return r.text



In [20]:
# Player Scraper
for year in range(1980,datetime.now().year + 1):

    df = pd.read_html(getdata(f'https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html'))[0]
    df_adv = pd.read_html(getdata(f'https://www.basketball-reference.com/leagues/NBA_{year}_advanced.html'))[0]


    df = df.drop(df[df['Player'] == 'Player'].index)


    df_adv = df_adv.drop(df_adv[df_adv['Player'] == 'Player'].index)


    new = pd.merge(df,df_adv,on=['Player','Age','Tm','Pos'])

    player_stats_list = ['Player', 'Pos', 'Age', 'Tm', 'G_x', 'GS', 'MP_x', 'FG', 'FGA',
           'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
           'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
            'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
           'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS',
           'WS', 'WS/48','OBPM', 'DBPM', 'BPM', 'VORP']

    player_stats_list_correct = ['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
           '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
           'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER',
           'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
           'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
           'VORP']

    # cuts down the columns to what is listed in player_stats_list_correct
    new = new[player_stats_list]
    new.columns = player_stats_list_correct

    # This gets rid of traded players entirely
    new = new.drop_duplicates(subset='Player',keep=False).reset_index(drop=True)

    # Fills nan's with 0
    new.fillna(value=0,inplace=True)

    # Adds a year column
    new['Year'] = year

    new.to_csv(f'player_stats/{year}_player_stats.csv')
    print(year,'Complete')

FileNotFoundError: [Errno 2] No such file or directory: 'player_stats/1980_player_stats.csv'

In [16]:
## Team Scraper


for year in range(1980,datetime.now().year + 1):

	htmldata = getdata(f"https://www.basketball-reference.com/leagues/NBA_{year}.html")
	df_list = pd.read_html(htmldata)

	per_game_stat = df_list[4].copy()

	if year <= 2015:
		adv_stat = df_list[8].copy()
	else:
		adv_stat = df_list[10].copy()

	new = pd.merge(per_game_stat,adv_stat.droplevel(level=0,axis=1),on='Team')

	new = new[[ 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
		   '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
		   'BLK', 'TOV', 'PF', 'PTS', 'Age', 'W', 'L', 'MOV',
		   'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%',]].sort_values('W',ascending=False).reset_index(drop=True)

	new['Year'] = year

	new.loc[new['Team'] == 'League Average','Team'] = f'League Average {year}'

	new.to_csv(f'team_stats/{year}_tm_stats.csv')
	print(year,'Complete')


FileNotFoundError: [Errno 2] No such file or directory: 'team_stats/1980_tm_stats.csv'